In [ ]:
%load_ext autoreload
%autoreload 2
from fibsem.structures import FibsemStagePosition, PatterningState
from fibsem.structures import FibsemRectangleSettings, FibsemCircleSettings, FibsemLineSettings
from fibsem import milling, patterning, utils
from pprint import pprint

from fibsem import utils

import logging
import time

CONFIGURATION_PATH = "/home/patrick/development/openfibsem/fibsem/fibsem/config/odemis-configuration.yaml"
microscope, settings = utils.setup_session(config_path=CONFIGURATION_PATH)

# pip install pylibtiff

In [ ]:
stage_position = microscope.get_stage_position()
print('Stage position:', stage_position)

In [ ]:
stage_position = microscope.get_stage_position()
print('Stage position:', stage_position)

microscope.move_stage_relative(FibsemStagePosition(x=50e-3))
stage_position = microscope.get_stage_position()
print('Stage position:', stage_position)

import time
time.sleep(2)

microscope.move_stage_absolute(FibsemStagePosition(x=0, y=0, z=0, r=0, t=0))
stage_position = microscope.get_stage_position()
print('Stage position:', stage_position)

In [ ]:
microscope.vertical_move(dx=0, dy=50e-6)

In [ ]:
from fibsem import acquire
from fibsem.structures import ImageSettings, BeamType, FibsemRectangle

settings.image.beam_type = BeamType.ELECTRON
settings.image.autocontrast = False
settings.image.reduced_area = FibsemRectangle(0.25, 0.25, 0.5, 0.5)
image = acquire.acquire_image(microscope, settings.image)

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(image.data, cmap='gray')
plt.show()


In [5]:
pattern_settings = FibsemRectangleSettings(
    centre_x=0, centre_y=0, width=10e-6, height=10e-6, 
    depth=0.5e-6, time=10
)
microscope.draw_rectangle(pattern_settings)

In [ ]:
microscope.run_milling(2e-9, 30e3, False)

In [ ]:
PROTOCOL_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/protocol/protocol-waffle.yaml"
protocol = utils.load_protocol(PROTOCOL_PATH)

stages = patterning.get_milling_stages("trench", protocol["milling"])

milling.mill_stages(microscope, stages)

In [ ]:
microscope.estimate_milling_time()

In [ ]:
# async milling
microscope.draw_rectangle(pattern_settings)
microscope.start_milling()

while microscope.get_milling_state() is PatterningState.RUNNING:
    time.sleep(5)
    logging.info(f'Milling...: {microscope.get_milling_state()}')

microscope.finish_milling(0, 0)

In [ ]:
circle_pattern = FibsemCircleSettings(
    centre_x=0, centre_y=0, radius=10e-6, depth=0.5e-6,
)
microscope.draw_circle(circle_pattern)

In [15]:
line_pattern = FibsemLineSettings(
    start_x=0, start_y=0, end_x=10e-6, end_y=10e-6, depth=0.5e-6
)
microscope.draw_line(line_pattern)

In [ ]:
microscope.estimate_milling_time()

In [7]:
microscope.run_milling(2e-9, 30e3, False)

In [ ]:
microscope.finish_milling(0, 0)

## AutoLamella-Odemis (Headless)

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils

from fibsem.structures import MicroscopeState
from autolamella.structures import Lamella, Experiment
from autolamella.waffle import run_autolamella
from pprint import pprint

from fibsem.microscopes.odemis_microscope import add_odemis_path
add_odemis_path()

from odemis.util.dataio import open_acquisition
from odemis import model
from pprint import pprint

CONFIGURATION_PATH = "/home/patrick/development/openfibsem/fibsem/fibsem/config/odemis-configuration.yaml"
# EXPERIMENT_PATH =  "/home/patrick/development/openfibsem/autolamella/autolamella/log/Odemis-Headless-AutoLamella-2024-09-23-15-56/experiment.yaml"
EXPERIMENT_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/log/AutoLamella-2024-09-30-11-45/experiment.yaml"
PROTOCOL_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/protocol/protocol-odemis-on-grid.yaml"

microscope, settings = utils.setup_session(config_path=CONFIGURATION_PATH, 
                                           protocol_path=PROTOCOL_PATH)

exp = Experiment.load(EXPERIMENT_PATH)
pprint(exp)

# headless mode (no gui, no ml)
# pyyaml, pandas, numpy==1.26.3
# pylibtiff for odemis


In [ ]:
microscope.vertical_move(dx=50e-6, dy=20e-6)

In [ ]:
run_autolamella(microscope, settings, exp, parent_ui=None)

In [ ]:
# requirements
# stage_position: FibsemStagePosition (stage-bare)                  -> acquire once position selected (can transform from fm-position)
# microcope_state: dict microscope state                            -> acquire once position selected
# reference_image: FibsemImage / OdemisImage (saved to disk)        -> acquire once position selected
# protocol: dict milling protocol for each lamella                  -> load from file, allow user to edit
# config: workflow configuration (supervision, alignment, etc)      -> load from file, allow user to edit

# TODO: load odemis-image as reference image
# TODO: draw milling stages in odemis
# TODO: CryoFeature -> Lamella
# TODO: protocol is not saved to same exp folder


# CryoFeature
# - stage_position
# - microscope_state
# - reference_image
# - milling_protocol
# - config

lamella = exp.positions[0]

print(lamella.state.microscope_state.stage_position)
print(lamella.state)
print(lamella.fiducial_area)
print(lamella.path)
print(lamella.info)
pprint(lamella.protocol)



# odemis: save_milling_position:
# - stage_position
# - microscope_state
# - reference_image
# - milling_protocol



# add to autoscript_client /  server
# vertical move



# req milling parameters
# center
# width
# height
# trench_height
# depth
# cross_section
# pattern_type

# only support non-fiducial milling for now


pprint(settings.protocol["options"])

In [ ]:
PATH = "/home/patrick/Pictures/20240729-450/test-image-FIBSEM-001.ome.tiff"

acq = open_acquisition(PATH)

md = acq[0].metadata[model.MD_EXTRA_SETTINGS]
ms = MicroscopeState.from_odemis_dict(md)
pprint(ms.stage_position)
pprint(ms.electron_beam)
pprint(ms.electron_detector)
pprint(ms.ion_beam)
pprint(ms.ion_detector)

In [ ]:
microscope.set_microscope_state(ms)

In [4]:
from datetime import datetime
now = datetime.now()
DATE = now.strftime("%Y-%m-%d-%H-%M")

from autolamella.config import EXPERIMENT_NAME

PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/log"
NAME = f"{EXPERIMENT_NAME}-{DATE}"

experiment = Experiment(path=PATH, name=NAME, program="AutoLamella", method="on-grid")
experiment.save()

print(experiment.path)
print(experiment.name)
print(experiment)

In [7]:
from copy import deepcopy
from fibsem.microscopes.odemis_microscope import add_odemis_path
from autolamella.compat.odemis import create_lamella_from_feature

add_odemis_path()

from odemis.acq.feature import CryoFeature, model
feature = CryoFeature(name="Feature-2", x=1e-6, y=20e-6, z=10e-6)
feature.stage_pos = model.VigilantAttribute({"x": 20e-6, "y": 50e-6, "z": 5e-6, "rx": 0.61, "rz": 0}, unit="m")

lamella =  create_lamella_from_feature(feature,
                                       path=experiment.path, 
                                       state=microscope.get_microscope_state(), 
                                       num=len(experiment.positions) + 1)

experiment.positions.append(deepcopy(lamella))
experiment.save()


In [ ]:
pprint(experiment)

In [ ]:
run_autolamella(microscope=microscope, 
                settings=settings, 
                experiment=experiment,
                parent_ui=None)

In [ ]:
t = 34.061224
print(f"{t:.2f}")

In [19]:
del experiment.positions[0]

In [ ]:
pprint(settings.protocol['options'])


In [ ]:
# supported options
# use_microexpansion:
# lamella_tilt_angle:
# num_polishing_steps: 

# not supported (for now)
# use_fiducial: only support full-frame alignment

In [ ]:
pprint(exp.positions[0].protocol)